In [1]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [2]:
from gensim import corpora

2017-02-28 04:45:45,773 : INFO : 'pattern' package not found; tag filters are not available for English


In [3]:
documents = ["Human machine interface for lab abc computer applications",
            "A survey of user opinion of computer system response time",
            "The EPS user interface management system",
            "System and human system engineering testing of EPS",
            "Relation of user perceived response time to error measurement",
            "The generation of random binary unordered trees",
            "The intersection graph of paths in trees",
            "Graph minors IV Widths of trees and well quasi ordering",
            "Graph minors A survey"]

In [5]:
stoplist = set('for a of the and to in'.split())
texts = [[word for word in document.lower().split() if word not in stoplist]
        for document in documents]

from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1
        
texts = [[token for token in text if frequency[token] > 1] for text in texts]

from pprint import pprint
pprint(texts)

[['human', 'interface', 'computer'],
 ['survey', 'user', 'computer', 'system', 'response', 'time'],
 ['eps', 'user', 'interface', 'system'],
 ['system', 'human', 'system', 'eps'],
 ['user', 'response', 'time'],
 ['trees'],
 ['graph', 'trees'],
 ['graph', 'minors', 'trees'],
 ['graph', 'minors', 'survey']]


In [6]:
dictionary = corpora.Dictionary(texts)
dictionary.save('/tmp/deerwester.dict')
print(dictionary)

2017-02-28 04:55:11,993 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2017-02-28 04:55:11,996 : INFO : built Dictionary(12 unique tokens: ['human', 'interface', 'computer', 'survey', 'user']...) from 9 documents (total 29 corpus positions)
2017-02-28 04:55:11,997 : INFO : saving Dictionary object under /tmp/deerwester.dict, separately None
2017-02-28 04:55:11,998 : INFO : saved /tmp/deerwester.dict


Dictionary(12 unique tokens: ['human', 'interface', 'computer', 'survey', 'user']...)


In [7]:
print(dictionary.token2id)

{'human': 0, 'interface': 1, 'computer': 2, 'survey': 3, 'user': 4, 'system': 5, 'response': 6, 'time': 7, 'eps': 8, 'trees': 9, 'graph': 10, 'minors': 11}


In [8]:
new_doc = "Human computer interaction"
new_vec = dictionary.doc2bow(new_doc.lower().split())
print(new_vec)

[(0, 1), (2, 1)]


In [9]:
corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize('/tmp/deerwester.mm', corpus)
for c in corpus:
    print(c)

2017-02-28 04:58:29,862 : INFO : storing corpus in Matrix Market format to /tmp/deerwester.mm
2017-02-28 04:58:29,865 : INFO : saving sparse matrix to /tmp/deerwester.mm
2017-02-28 04:58:29,865 : INFO : PROGRESS: saving document #0
2017-02-28 04:58:29,866 : INFO : saved 9x12 matrix, density=25.926% (28/108)
2017-02-28 04:58:29,868 : INFO : saving MmCorpus index to /tmp/deerwester.mm.index


[(0, 1), (1, 1), (2, 1)]
[(2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)]
[(1, 1), (4, 1), (5, 1), (8, 1)]
[(0, 1), (5, 2), (8, 1)]
[(4, 1), (6, 1), (7, 1)]
[(9, 1)]
[(9, 1), (10, 1)]
[(9, 1), (10, 1), (11, 1)]
[(3, 1), (10, 1), (11, 1)]


In [10]:
class MyCorpus(object):
    def __iter__(self):
        for line in open('datasets/mycorpus.txt'):
            yield dictionary.doc2bow(line.lower().split())

In [11]:
corpus_memory_friendly = MyCorpus()
print(corpus_memory_friendly)

In [13]:
for vector in corpus_memory_friendly:
    print(vector)

[(0, 1), (1, 1), (2, 1)]
[(2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)]
[(1, 1), (4, 1), (5, 1), (8, 1)]
[(0, 1), (5, 2), (8, 1)]
[(4, 1), (6, 1), (7, 1)]
[(9, 1)]
[(9, 1), (10, 1)]
[(9, 1), (10, 1), (11, 1)]
[(3, 1), (10, 1), (11, 1)]


In [14]:
from six import iteritems

dictionary = corpora.Dictionary(line.lower().split() for line in open('datasets/mycorpus.txt'))

stop_ids = [dictionary.token2id[stopword] for stopword in stoplist
           if stopword in dictionary.token2id]
once_ids = [tokenid for tokenid, docfreq in iteritems(dictionary.dfs) if docfreq == 1]

dictionary.filter_tokens(stop_ids + once_ids)

dictionary.compactify()
print(dictionary)

2017-02-28 05:10:52,256 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2017-02-28 05:10:52,259 : INFO : built Dictionary(42 unique tokens: ['human', 'machine', 'interface', 'for', 'lab']...) from 9 documents (total 69 corpus positions)


Dictionary(12 unique tokens: ['human', 'interface', 'computer', 'survey', 'user']...)


In [15]:
corpus = [[(1, 0.5)], []]

corpora.MmCorpus.serialize('/tmp/corpus.mm', corpus)

2017-02-28 05:12:08,197 : INFO : storing corpus in Matrix Market format to /tmp/corpus.mm
2017-02-28 05:12:08,199 : INFO : saving sparse matrix to /tmp/corpus.mm
2017-02-28 05:12:08,201 : INFO : PROGRESS: saving document #0
2017-02-28 05:12:08,202 : INFO : saved 2x2 matrix, density=25.000% (1/4)
2017-02-28 05:12:08,204 : INFO : saving MmCorpus index to /tmp/corpus.mm.index


In [16]:
corpora.SvmLightCorpus.serialize('/tmp/corpus.svmlignt', corpus)
corpora.BleiCorpus.serialize('/tmp/corpus.lda-c', corpus)
corpora.LowCorpus.serialize('/tmp/corpus.low', corpus)

2017-02-28 06:30:47,656 : INFO : converting corpus to SVMlight format: /tmp/corpus.svmlignt
2017-02-28 06:30:47,667 : INFO : saving SvmLightCorpus index to /tmp/corpus.svmlignt.index
2017-02-28 06:30:47,670 : INFO : no word id mapping provided; initializing from corpus
2017-02-28 06:30:47,672 : INFO : storing corpus in Blei's LDA-C format into /tmp/corpus.lda-c
2017-02-28 06:30:47,673 : INFO : saving vocabulary of 2 words to /tmp/corpus.lda-c.vocab
2017-02-28 06:30:47,674 : INFO : saving BleiCorpus index to /tmp/corpus.lda-c.index
2017-02-28 06:30:47,676 : INFO : no word id mapping provided; initializing from corpus
2017-02-28 06:30:47,677 : INFO : storing corpus in List-Of-Words format into /tmp/corpus.low
2017-02-28 06:30:47,678 : WARNING : List-of-words format can only save vectors with integer elements; 1 float entries were truncated to integer value
2017-02-28 06:30:47,679 : INFO : saving LowCorpus index to /tmp/corpus.low.index


In [17]:
corpus = corpora.MmCorpus('/tmp/corpus.mm')

2017-02-28 06:31:25,465 : INFO : loaded corpus index from /tmp/corpus.mm.index
2017-02-28 06:31:25,466 : INFO : initializing corpus reader from /tmp/corpus.mm
2017-02-28 06:31:25,467 : INFO : accepted corpus with 2 documents, 2 features, 1 non-zero entries


In [18]:
print(corpus)

MmCorpus(2 documents, 2 features, 1 non-zero entries)


In [19]:
print(list(corpus))

[[(1, 0.5)], []]


In [20]:
for doc in corpus:
    print(doc)

[(1, 0.5)]
[]


In [21]:
corpora.BleiCorpus.serialize('/tmp/corpus.lda-c', corpus)

2017-02-28 06:32:49,714 : INFO : no word id mapping provided; initializing from corpus
2017-02-28 06:32:49,716 : INFO : storing corpus in Blei's LDA-C format into /tmp/corpus.lda-c
2017-02-28 06:32:49,718 : INFO : saving vocabulary of 2 words to /tmp/corpus.lda-c.vocab
2017-02-28 06:32:49,723 : INFO : saving BleiCorpus index to /tmp/corpus.lda-c.index


In [22]:
import gensim
import numpy as np
numpy_matrix = np.random.randint(10, size=[5,2])
corpus = gensim.matutils.Dense2Corpus(numpy_matrix)
numpy_matrix_dense = gensim.matutils.corpus2dense(corpus, num_terms=10)

In [23]:
import scipy.sparse
scipy_sparse_matrix = scipy.sparse.random(5,2)
corpus = gensim.matutils.Sparse2Corpus(scipy_sparse_matrix)
scipy_csc_matrix = gensim.matutils.corpus2csc(corpus)